In [35]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell, seq
import re

In [21]:
re.findall("^-?[0-9\.]+$|[\(\)]", "1.234.23(sdlk)")
re.sub("[/\\\\]", " ", "sp/id/e\\r")
re.findall("[0-9]{1,}", ", 1, 23, 456, 7890")

['1', '23', '456', '7890']

In [44]:
def normalize(line):
    line = line.lower()
    line = re.sub("[-/\\\\]", " ", line)
    line = re.sub("[^-\w\d \.\t]", " ", line)
    line = re.sub("  +", " ", line)
    return line

def parseLine(line):
    vals = line.strip("\n ").split("\t")
    query, title = vals[0], vals[2]
    return (query, title)

def queryTitleFilter(query, title):
    negative_match = "-\(|^-\w| -\w"
    chars_to_reject = "[\(\)\[\]\"]"
    num_only = "^-?[0-9\.]+$"
    any_number = "-?[0-9\.]{2,}"
    if len(query.split(" ")) >= 5 or len(title.split(" ")) >= 30:
        return False
    if len(query.split(" ")) < 1 or len(query.strip()) == 0:
        return False
    if len(re.findall("|".join([negative_match, chars_to_reject, num_only, any_number]), query)) > 0:
        return False   
    return True

def writeLines(ar, filename):
    with open(filename, "w") as out:
         for i, x in enumerate(ar):
            print(x, file=out)
            

queryTitlePairsRaw = (parseLine(line) for line in open("/home/ubuntu/data/queryClickPairs/part-00000.tsv", "r") )
queryTitlePairs = ((normalize(q),normalize(t)) for q,t in queryTitlePairsRaw if queryTitleFilter(q,t))
#queryTitlePairs = ((q,t) for q,t in queryTitlePairsRaw if queryTitleFilter(q,t))

In [47]:
lines = [next(queryTitlePairs) for _ in range(1000000)]
lines_train = [next(queryTitlePairs) for _ in range(100000)]
lines_dev = [next(queryTitlePairs) for _ in range(100000)]

In [48]:
writeLines([q for q,t in lines], "/home/ubuntu/data/query_titles/query.txt")
writeLines([t for q,t in lines], "/home/ubuntu/data/query_titles/title.txt")
writeLines([q for q,t in lines_train], "/home/ubuntu/data/query_titles/query.train")
writeLines([t for q,t in lines_train], "/home/ubuntu/data/query_titles/title.train")
writeLines([q for q,t in lines_dev], "/home/ubuntu/data/query_titles/query.dev")
writeLines([t for q,t in lines_dev], "/home/ubuntu/data/query_titles/title.dev")